In [3]:
from urlparse import urljoin
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
from unidecode import unidecode

In [5]:
def remove_non_ascii(text):
    return unidecode(text)

In [10]:
def extract_album_information(description):
    album_year = ''
    year_index = 0
    
    for i, word in enumerate(description):
        year_match = re.match(r'.*\((\d{4})\).*', word)
        if year_match:
            album_year = year_match.group(1)
            year_index = i
            break
            
    album_title = remove_non_ascii(' '.join(description[:i]))
    
    if album_year: album_year = int(album_year)
    
    return album_title, album_year

In [22]:
non_song_words = re.compile(r'(?i)\[.*\]|\(single|tracklist|album art|trailer')
if non_song_words.search('kasaasdf kasl;dfj Album Art'):
    print('match')

match


In [23]:
def extract_lyrics(song_soup):
    forbidden_words = re.compile(r'(?i)intro|outro|skit|interlude|live')
    non_song_words = re.compile(r'(?i)\[.*\]|\(single|tracklist|album art|trailer')
    title = song_soup.find("span", {"ng-bind":"song.title"}).text
    if forbidden_words.search(title) or non_song_words.search(title):
        return None, title
    
    lyrics = []
    lyrics_raw = soup.find(class_="lyrics")
    paragraphs = lyrics_raw.text.split('\n\n')
    
    for paragraph in paragraphs:
        lines = paragraph.split('\n')
        empty_line = re.compile(r'^\s*$')
        brackets = re.compile(r'\s*[\(\[].*[\)\]].*')
        verse = re.compile(r'(?i)verse\s*\d+')
        lines = [remove_non_ascii(line) for line in lines 
                 if not (empty_line.search(line) or 
                         brackets.search(line) or 
                         verse.search(line) or
                         len(line.split()) <=1)]
        for i, line in enumerate(lines):
            line = re.sub(r'\{.*\}', '', line)
            if i + 1 < len(lines):
                lyrics.append((line, lines[i + 1]))
            else:
                lyrics.append((line, None))

    return lyrics, remove_non_ascii(title)

In [12]:
def parse_lyrics(lyrics, artist, title, album, year):
    lines_data = []
    lines = []
    
    for line, next_line in lyrics:
        if line not in lines:
            lines.append(line)
            
            line_dict = {}
            line_dict['line'] = line
            line_dict['next_line'] = next_line
            line_dict['artist'] = artist
            line_dict['title'] = title
            line_dict['album'] = album
            line_dict['year'] = year
            
            lines_data.append(line_dict)
            
    pd.DataFrame(lines_data).to_csv('lyrics.csv', mode='a', header=False, index=False)

In [24]:
df = pd.read_csv('Rappers.csv')
for artist in df.Artist[50:]:
    artist_url = "https://genius.com/artists/" + artist
    artist_response = requests.get(artist_url, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 6.3; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/56.0.2924.87 Safari/537.36'})
    soup = BeautifulSoup(artist_response.text, 'html5lib')
    albums = soup.find_all("a", class_="vertical_album_card", href=True)
    print('Artist Found: ' + artist)
    for album_raw in albums:
        album_response = requests.get(album_raw['href'])
        soup = BeautifulSoup(album_response.text, 'html5lib')
        album_description = soup.find(class_="album_name").text.split()
        album_title, album_year = extract_album_information(album_description)
        print('Album Found: {0} ({1})'.format(album_title, album_year))
        for song_raw in soup.find_all("a", class_="song_name", href=True):
            song_response = requests.get(song_raw['href'])
            soup = BeautifulSoup(song_response.text, 'html5lib')
            lyrics, title = extract_lyrics(soup)
            if lyrics:
                print('\t Song Found: {0} ({1} lines)'.format(title, len(lyrics)))
                parse_lyrics(lyrics, artist, title, album_title, album_year)

Artist Found: Rakim
Album Found: The 18th Letter (1997)
	 Song Found: The 18th Letter (Always and Forever) (45 lines)
	 Song Found: It's Been A Long Time (51 lines)
	 Song Found: Remember That (57 lines)
	 Song Found: The Saga Begins (58 lines)
	 Song Found: Guess Who's Back (77 lines)
	 Song Found: Stay Awhile (55 lines)
	 Song Found: New York (Ya Out There?) (63 lines)
	 Song Found: Show Me Love (50 lines)
	 Song Found: Mystery (Who Is God) (77 lines)
	 Song Found: When I'm Flowin (58 lines)
	 Song Found: It's Been A Long Time (Suave House Mix) (48 lines)
	 Song Found: Guess Who's Back (Alternative Mix) (66 lines)
	 Song Found: I Know You Got Soul (103 lines)
	 Song Found: Let The Rhythm Hit 'Em (94 lines)
	 Song Found: In the Ghetto (84 lines)
Album Found: The Master (1999)
	 Song Found: Flow Forever (60 lines)
	 Song Found: When I B on the Mic (80 lines)
	 Song Found: Finest Ones (63 lines)
	 Song Found: All Night Long (57 lines)
	 Song Found: Uplift (69 lines)
	 Song Found: I Know